In [1]:
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [2]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard


2.15.0


In [3]:
X = X/255.0 # normalized data

In [6]:
activation = 'sigmoid'
feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = X.shape[1:]))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Flatten())

In [7]:
x = feature_extractor.output  
x = Dense(128, activation = activation, kernel_initializer = 'he_uniform')(x)
prediction_layer = Dense(5, activation = 'softmax')(x)

In [8]:
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer='rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 128, 128, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 batch_normalization_1 (Bat  (None, 128, 128, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0    

In [9]:
cnn_model.fit(X, y, epochs=75)

Epoch 1/75


9/9 [==============================] - 9s 802ms/step - loss: 1.7591 - accuracy: 0.3394
Epoch 2/75
9/9 [==============================] - 8s 848ms/step - loss: 1.0000 - accuracy: 0.5993
Epoch 3/75
9/9 [==============================] - 7s 788ms/step - loss: 0.8177 - accuracy: 0.6390
Epoch 4/75
9/9 [==============================] - 6s 687ms/step - loss: 1.0319 - accuracy: 0.5993
Epoch 5/75
9/9 [==============================] - 6s 720ms/step - loss: 0.7796 - accuracy: 0.6931
Epoch 6/75
9/9 [==============================] - 7s 784ms/step - loss: 0.7131 - accuracy: 0.7401
Epoch 7/75
9/9 [==============================] - 7s 800ms/step - loss: 0.7056 - accuracy: 0.7365
Epoch 8/75
9/9 [==============================] - 7s 822ms/step - loss: 0.6677 - accuracy: 0.7329
Epoch 9/75
9/9 [==============================] - 7s 820ms/step - loss: 0.6304 - accuracy: 0.7581
Epoch 10/75
9/9 [==============================] - 7s 751ms/step - loss: 0.5628 - accuracy: 0.8014
Epoch 11/75
9/9 [

In [10]:
x_train_RF = feature_extractor.predict(X)

9/9 [==============================] - 2s 175ms/step


In [11]:
print(x_train_RF.shape)

(277, 65536)


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [13]:
parameters = {'n_estimators': (10,20,30,40,50,70), 'criterion': ('gini', 'entropy'), 'max_depth': (3,5,7),
             'max_features' : (2,4), 'min_samples_split' : (2,4)}

In [14]:
RF_grid = GridSearchCV(RandomForestClassifier(n_jobs=-1, oob_score=False), param_grid=parameters, cv=3, verbose=True)

In [15]:
RF_grid_model = RF_grid.fit(x_train_RF, y)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


In [16]:
RF_grid_model.best_estimator_

RandomForestClassifier(max_depth=7, max_features=2, min_samples_split=4,
                       n_estimators=70, n_jobs=-1)

In [17]:
RF_grid_model.best_score_

0.906069814555088

In [18]:
RF_model = RandomForestClassifier(max_depth=7, max_features=4, n_estimators=40, n_jobs=-1)

In [19]:
RF_model.fit(x_train_RF, y)

RandomForestClassifier(max_depth=7, max_features=4, n_estimators=40, n_jobs=-1)

In [26]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.


In [27]:
feature_extractor.save('feature_extractor_model.h5')
from joblib import dump, load
dump(RF_model, 'RF_model.joblib')

C:\Users\adria\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['RF_model.joblib']